In [1]:
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlite3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, Table, MetaData

# Reflect Tables into SQLAlchemy ORM

In [10]:
# create engine to min_wage.db
engine = create_engine(r'sqlite:///../min_wage.sqlite')

# reflect the tables
meta = MetaData()
meta.reflect(engine)

Base = automap_base(metadata=meta)

Base.prepare()
inspector = inspect(engine)

print(inspector.get_table_names())



# Save references to each table
wages_tbl = Table('wages', meta, autoload=engine)
pce_tbl = Table('pce', meta, autoload=engine)
rpp_tbl = Table('rpp', meta, autoload=engine)


columns = inspector.get_columns('spce')
for c in columns:
    print(c['name'])
pce_tbl
Base.classes.rpp

['cat_pce', 'cat_states', 'cat_years', 'pce', 'rpp', 'wages']


AttributeError: rpp

In [4]:
Base.classes.wages

AttributeError: wages

In [3]:
# Create session (link) from Python to the DB
session = Session(bind=engine)

In [5]:
pce_data = session.query(pce_tbl.c.Year,pce_tbl.c.state_name,pce_tbl.c.pce_category,pce_tbl.c.pce_value)
df = pd.DataFrame(pce_data)
df

,0,1,2,3
0,1997,United States,Clothing and footwear,908
1,1997,United States,Gasoline and other energy goods,542
2,1997,United States,Household consumption expenditures (for services),12561
3,1997,United States,Housing and utilities,3704
4,1997,United States,Health care,2901
...,...,...,...,...
5975,2019,Wyoming,Clothing and footwear,878
5976,2019,Wyoming,Gasoline and other energy goods,2352
5977,2019,Wyoming,Household consumption expenditures (for services),29090
5978,2019,Wyoming,Housing and utilities,7877


# Close Session

In [ ]:
# Close Session
session.close()